### Importaciones

In [353]:
# Paquetes necesarios para la ejecución del notebook
import win32com.client
import os
from datetime import datetime
import pandas as pd
import shutil
import openpyxl
from openpyxl import load_workbook
from openpyxl.workbook import Workbook
from copy import copy
from openpyxl.utils import get_column_letter, column_index_from_string
import polars as pl

### Configuracion

In [354]:
# Constantes usadas en el notebook
MAPI = "MAPI" # Messaging Application Programming Interface
DOT = "."
OUTLOOK = "Outlook"
APPLICATION = "Application"
MAIL_ITEM_CODE = 43

# Diccionarios
outlook_folder_codes = {
    0: 'Calendario',
    1: 'Contactos',
    2: 'Borradores',
    3: 'Diario / Jornal',
    4: 'Notas',
    5: 'Tareas',
    6: 'Bandeja de entrada',
    7: 'Bandeja de salida',
    8: 'Elementos enviados',
    9: 'Elementos eliminados',
    10: 'Bandeja de correo del servidor',
    11: 'Conflictos',
    12: 'Elementos de sincronizacion local',
    13: 'Elementos de sincronizacion (Envio)',
    14: 'Elementos de sincronización (Recibo)',
    15: 'Elementos de sincronización completa',
    16: 'Diario de formularios',
    17: 'Carpeta de búsqueda',
    18: 'Bandeja para reglas cliente',
    19: 'Carpeta de sugerencias de correo',
}
parse_locaciones = {
    '06 AYA EL PEDREGAL': 'El Pedregal',
    '38 AYA ATICO': 'Atico',
    '40 AYA CHALA': 'Chala',
    '88 AYA CAMANA': 'Camana'
}
meta = {
    '06 AYA EL PEDREGAL': {
        2023: {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: 0.7, 6: 0.7, 7: 0.7, 8:0.7, 9:0.7, 10:0.7, 11:0.7, 12:0.7},
        2024: {1: 1.2, 2: 1.2, 3: 1.2, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1},
        2025: {1: 0.88, 2: 0.88, 3: 0.88, 4: 0.88, 5: 0.88, 6: 0.88, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0},
    },
    '38 AYA ATICO': {
        2023: {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: 0.7, 6: 0.7, 7: 0.7, 8:0.7, 9:0.7, 10:0.7, 11:0.7, 12:0.7},
        2024: {1: 0.4, 2: 0.4, 3: 0.4, 4: 0.48, 5: 0.48, 6: 0.48, 7: 0.48, 8: 0.48, 9: 0.48, 10: 0.48, 11: 0.48, 12: 0.48},
        2025: {1: 0.24, 2: 0.24, 3: 0.24, 4: 0.24, 5: 0.24, 6: 0.24, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0},
    },
    '40 AYA CHALA': {
        2023: {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: .7, 6: 0.7, 7: 0.7, 8:0.7, 9:0.7, 10:0.7, 11:0.7, 12:0.7},
        2024: {1: 0.5, 2: 0.5, 3: 0.5, 4: 0.60, 5: 0.60, 6: 0.60, 7: 0.60, 8: 0.60, 9: 0.60, 10: 0.60, 11: 0.60, 12: 0.60},
        2025: {1: 0.31, 2: 0.31, 3: 0.31, 4: 0.31, 5: 0.31, 6: 0.31, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0},
    },
    '88 AYA CAMANA': {
        2023: {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: 0.7, 6: 0.7, 7: 0.7, 8:0.7, 9:0.7, 10:0.7, 11:0.7, 12:0.7},
        2024: {1: 0.85, 2: 0.85, 3: 0.85, 4: 0.85, 5: 0.85, 6: 0.85, 7: 0.85, 8: 0.85, 9: 0.85, 10: 0.85, 11: 0.85, 12: 0.85},
        2025: {1: 0.57, 2: 0.57, 3: 0.57, 4: 0.57, 5: 0.57, 6: 0.57, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0},
    }
}
vendedores = {
    '06 AYA EL PEDREGAL': {
        '0000001013': 'AYALA CCAHUANA EDMUNDO',
        '0000001006': 'BARRAZA REVILLA FREDY',
        '0000001000': 'CARDENAS CHOQUECAJIA WHITNEY AWARD',
        '0000001001': 'CHARA ROJAS GISELA SOLEDAD',
        '0000001007': 'CHINO QUISPE JORGE LUIS',
        '0000006014': 'EDWAR QUISPE CHINO',
        '0000006009': 'GALLARDO HUAYLLA RENATO LEUTERIO',
        '0000006003': 'GALLEGOS NUÑEZ JUAN FREDY',
        '0000005999': 'GESTOR VIRTUAL',
        '0000001010': 'HUANCA MAMANI JACKSON',
        '0000006006': 'HUAYCHO TORRES ULBER',
        '0000006005': 'HUAYCHO TORRES ULBER',
        '0000001012': 'HUAYLLAZO HUACCHA CRISTHIAN PAOLO',
        '0000006012': 'NELSON RAUL CONZA CHARCA',
        '0000001002': 'PACHECO CONDORI ANDRES OSCAR',
        '0000001011': 'QUISPE CHINO WILIAN YURI',
        '0000001004': 'QUISPE HUAYLLA MARISOL S.',
        '0000006013': 'RONALD GONZALO HUILLCA MAMANI',
        '0000001003': 'TACO XESSPE KELY SOFIA',
        '0000001005': 'VALDERRAMA ELLIS JESSICA A.',
        '0000001008': 'VEND NVA RUTA 10',
        '0000001009': 'YUCRA JIMENEZ ANA LUZ',
        '0000001014': 'CHARA ROJAS GISELA SOLEDAD',
        '0000001015': 'BARRAZA REVILLA FREDY',
        '0000001019': 'BATALLANOS SANCA RODRIGO LEOPOLDO',
        '0000001016': 'PARI PUCHO FREDY OSWALDO',
        '0000001017': 'COAGUILLA MAMANI JOSE ALBERTO',
        '0000001018': 'ORTEGA MAMANI JORGE LUIS'
    },
    '38 AYA ATICO': {
        '0000001001': 'ALANYA RAMIREZ FERNANDO JOSUE',
        '0000005999': 'GESTOR VIRTUAL',
        '0000001000': 'SAUL ANDRES VIÑA VIZCARDO'
    },
    '40 AYA CHALA': {
        '0000001004': 'CANALES AGUILAR HILBERTO',
        '0000001007': 'CHIPANA JURADO JHORS EDUARDO',
        '0000006010': 'DIONICIO DANIEL HUARCAYA SALAZAR',
        '0000005999': 'GESTOR VIRTUAL',
        '0000006009': 'GLOBER FELIPE JARA MAQUER',
        '0000006005': 'GONZALES CHURA MAGNO ALFREDO',
        '0000001000': 'HERRERA TAPIA EDWIN DONATO',
        '0000006002': 'JARA CARAZAS RODOLFO JOSUE.L',
        '0000006007': 'JUAN CARLOS ARIAS BENITES',
        '0000001003': 'MAMANI TINTAYA KRISTHOFER',
        '0000001006': 'MAMANI TINTAYA KRISTHOFER',
        '0000001005': 'QUISE CCAPA EVER',
        '0000001002': 'RODRIGUEZ JOSE ANTONIO',
        '0000001001': 'VENDEDOR RT M1'        
    },
    '88 AYA CAMANA': {
        '0000001007': 'AUCAHUAQUI REVILLA DANIEL HITLER',
        '0000001011': 'CARAZAS REZA LUIS ALBERTO',
        '0000001003': 'CONDORCHOA SIERRA NEMECIO',
        '0000006001': 'CONDORCHOA SIERRA NEMESIO JESUS',
        '0000006002': 'DE LA CRUZ CALCINA ELAR',
        '0000006010': 'EDILBERTO RAMIREZ LARICO',
        '0000005999': 'GESTOR VIRTUAL',
        '0000001002': 'HUAMANI RODRIGUEZ YONATAN ANYIMZAN',
        '0000006011': 'JAIME CHAVEZ CONDORI',
        '0000001005': 'LLERENA DE LA CRUZ RICARDO SNEIDER',
        '0000001004': 'MEDINA VELASQUEZ JAVIER ENRIQUE',
        '0000006014': 'MOISES RICHARD CONDORCHOA SIERRA',
        '0000001008': 'MOLLO YUPANQUI JOSE OMAR',
        '0000001001': 'NO APLICA VENDEDOR',
        '0000006008': 'QUISPE CCACHUCO FREDY',
        '0000001010': 'RAMOS MAMANI RUBEN',
        '0000006016': 'RENATO ELEUTERIO GALLARADO HUAYLLA',
        '0000001006': 'SACARI CHOQUEHUANCA WILSON',
        '0000001000': 'SALAZAR HUAMANI SAUL ANTONIO',
        '0000001012': 'VEND RT X1',
        '0000001009': 'VIZCARDO BUSTAMANTE ALBERTH FRANCK'
    }
}

# Listas
locaciones = ['06 AYA EL PEDREGAL', '38 AYA ATICO', '40 AYA CHALA', '88 AYA CAMANA']

# Variables de configuracion
root_address = r'C:\Informacion\rechazos' # Direcccion de carpeta raiz
test_address = r'\prueba'
backup_address = r'\backup'
transportista = {
    "mail_subject": "Reporte de ordenes de carga diario",         # Nombre del asunto de correo
    "local_file_name": "Cf_programadas_por_transportista.csv",    # Nombre del archivo local
    "mail_file_address": "",                                      # Direccion del archivo en el correo
    "mail_sheet_name": "Guias",
    "mail_received_time": "",
    "date": "Fecha",
    "date_format": "dd\/mm\/yyyy",
}
ruta = {
    "mail_subject": "Venta perdida diaria por cliente y ruta - acum mes",   # Nombre del asunto de correo
    "local_file_name": "Cf_rech_por_ruta.csv",                              # Nombre del archivo local
    "mail_file_address": "",                                                # Direccion del archivo en el correo
    "mail_sheet_name": "Motivos_VP (clte)",
    "mail_received_time": "",
    "date": "Día",
    "date_format": "d\/m\/yy",
}

<>:150: SyntaxWarning: invalid escape sequence '\/'
<>:159: SyntaxWarning: invalid escape sequence '\/'
<>:150: SyntaxWarning: invalid escape sequence '\/'
<>:159: SyntaxWarning: invalid escape sequence '\/'
C:\Users\AYACDA23\AppData\Local\Temp\ipykernel_13424\2964206209.py:150: SyntaxWarning: invalid escape sequence '\/'
  "date_format": "dd\/mm\/yyyy",
C:\Users\AYACDA23\AppData\Local\Temp\ipykernel_13424\2964206209.py:159: SyntaxWarning: invalid escape sequence '\/'
  "date_format": "d\/m\/yy",


### Obtener correos de outlook

In [355]:
# Conectar a Outlook
#outlook_folder_code = int(input(f'{" ".join(["(" + str(key) + ": " + value + ")" for key, value in outlook_folder_codes.items()])}'))
outlook = win32com.client.Dispatch(OUTLOOK+DOT+APPLICATION).GetNamespace(MAPI)

outlook_folder = outlook.GetDefaultFolder(6)
print("Tipo de folder: ", outlook_folder)

Tipo de folder:  Bandeja de entrada


### Guardar archivo de outlook

In [356]:
# Buscar el correo más reciente con archivo Excel
mails = outlook_folder.Items

# Ordenar por fecha descendente
mails.Sort("[ReceivedTime]", True) # (mails) Es un objeto lista

''' MAIL PROPERTIES
    mail.Subject
    mail.ReceivedTime
    mail.SenderName
    mail.SenderEmailAddress
    mail.To
    mail.CC
    mail.Body
    mail.Attachments.Count
    mail.CreationTime
    mail.LastModificationTime
    mail.EntryID
'''

def download_excel_file(mails, mail_subject):
    for mail in mails: # Recorrer todos los correos
        if mail.Class != MAIL_ITEM_CODE: # Asegurar que el correo sea un MailItem (otros pueden ser calendario, alertas, etc)
            continue
        if (mail_subject.lower() in mail.Subject.lower()) & (mail.Attachments.Count > 0):
            for attachment in mail.Attachments:
                if attachment.FileName.endswith((".xlsx", ".xls")):
                    file_name = attachment.FileName
                    file_address = os.path.join(root_address+test_address, file_name)
                    attachment.SaveAsFile(file_address)
                    return file_address, mail.ReceivedTime

transportista['mail_file_address'], transportista['mail_received_time'] = download_excel_file(mails, transportista['mail_subject'])
ruta['mail_file_address'], ruta['mail_received_time'] = download_excel_file(mails, ruta['mail_subject'])

for x,y in transportista.items():
    print(f'{x}: {y}')
print('\n')
for x,y in ruta.items():
    print(f'{x}: {y}')

mail_subject: Reporte de ordenes de carga diario
local_file_name: Cf_programadas_por_transportista.csv
mail_file_address: C:\Informacion\rechazos\prueba\Reporte de Guias de Distribución por FechaV3_934_3540206480547260789.xlsx
mail_sheet_name: Guias
mail_received_time: 2025-05-30 09:01:04+00:00
date: Fecha
date_format: dd\/mm\/yyyy


mail_subject: Venta perdida diaria por cliente y ruta - acum mes
local_file_name: Cf_rech_por_ruta.csv
mail_file_address: C:\Informacion\rechazos\prueba\Venta perdida x Cliente y ruta diaria_725_605084156184953594.xlsx
mail_sheet_name: Motivos_VP (clte)
mail_received_time: 2025-05-29 08:18:00+00:00
date: Día
date_format: d\/m\/yy


### Generar csv

In [357]:
def delete_unnamed_columns(df):
    # Eliminar columnas llamadas Unnamed
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
        
    return df

def clean_mail_file(df):
    # Eliminar columnas unnamed (vacias)
    df = df.dropna(axis=1, how='all')

    # Tomar la fila 1 como nombres de columnas
    df.columns = df.iloc[1]

    # Eliminar las dos primeras filas (la original de encabezado y la fila de nombres)
    df = df.iloc[2:].reset_index(drop=True)
    
    return df

def filter_mail_file_locations(df):
    # Conservar las filas con locaciones especificas
    df = df[df['Locación'].isin(locaciones)]

    return df

def filter_mail_file_dates(document, df_mail, df_local):
    df_local_copy = df_local.copy()

    # Convertir columna 'Fecha' a datetime en archivo recibido
    df_local_copy[document['date']+"2"] = pd.to_datetime(df_local_copy[document['date']], dayfirst=True, errors='coerce')

    # Fecha máxima en archivo local
    fecha_max_local = df_local_copy[document['date']+"2"].max()

    # Filtrar solo filas con fecha mayor que la máxima del local
    df_mail = df_mail[df_mail[document['date']] > fecha_max_local].copy()

    return df_mail

def customized_mail_file(df):
    # Personalizada 1: Eliminar ultima columna
    if 'Mesa Comercial' in df.columns:
        df = df.drop(columns=['Mesa Comercial'])

    return df


In [ ]:
def update_local_file(document):
    # Rutas de archivo
    mail_file_address = document['mail_file_address']
    mail_sheet_name = document['mail_sheet_name']
    local_file_address = os.path.join(root_address+test_address, document['local_file_name'])

    # Leer datos del archivo local
    df_local = pd.read_csv(local_file_address, sep=';')
    df_local = delete_unnamed_columns(df_local)

    # Leer datos del archivo de correo
    df_mail = pd.read_excel(mail_file_address, sheet_name=mail_sheet_name, header=None)
    df_mail = clean_mail_file(df_mail)
    df_mail = filter_mail_file_locations(df_mail)
    print(len(df_mail))

    # Filtrar por fecha
    df_mail = filter_mail_file_dates(document, df_mail, df_local)
    print(len(df_mail))

    # Asegurar los mismos tipos de datos (str, int, ...)
    # df_mail.columns = df_local.columns
    # df_mail = df_mail.astype(df_local.dtypes.to_dict())

    # # Convertir la fecha a string, incluso si los valores son datetime dentro de "object"
    # df_mail[document['date']] = df_mail[document['date']].apply(lambda x: x.strftime('%d/%m/%Y') if isinstance(x, pd.Timestamp) or isinstance(x, datetime) else str(x))
    # #print(document['date'].apply(type).value_counts())

    # # Convertir a Polars
    # df_local = pl.from_pandas(df_local)
    # df_mail = pl.from_pandas(df_mail)

    # if df_mail.schema != df_local.schema:
    #     print("⚠️ Los esquemas no coinciden")

    # print(f'Local: {df_local.schema}')
    # print(df_local.shape)
    # print(f'Mail: {df_mail.schema}')
    # print(df_mail.shape)

    # # Combinar los datos (sin eliminar duplicados)
    # df_updated = pl.concat([df_local, df_mail])

    # print(f'Updated: {df_mail.schema}')
    # print(df_updated.shape)
    # print(df_updated)

    # # Escribir al CSV, sobrescribiendo el original
    # df_updated = df_updated.write_csv(separator=";")
    # with open(local_file_address, "w", encoding="utf-8-sig") as f:
    #     f.write(df_updated)

#update_local_file(transportista)
update_local_file(ruta)

c:\Users\AYACDA23\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11351 entries, 0 to 11350
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Tipo de Reparto                 11351 non-null  object 
 1   Código Locación                 11351 non-null  int64  
 2   Locación                        11351 non-null  object 
 3   Código de Transportista         11351 non-null  int64  
 4   Transportista                   11351 non-null  object 
 5   Orden de Carga                  11350 non-null  float64
 6   Nro Viaje                       11350 non-null  float64
 7   Placa de Vehículo               11350 non-null  object 
 8   Guía de Transportista           11350 non-null  float64
 9   Fecha                           11351 non-null  object 
 10  Carga Total CF                  11350 non-null  float64
 11  Redondeo CF                     11350 non-null  float64
 12  CF Programada                   